In [1]:
#! /usr/bin/env python
# -*-coding:utf-8-*-
# Author: Ming Chen
# create date: 2019-11-27 10:36:59
# description: 使用HMM进行命名实体识别NER
# 包括人名PER,地名LOC机构名ORG,其他O


import numpy as np
from tqdm import tqdm

In [2]:

class HMM_model:
    def __init__(self):
        self.n_tag = 7  # 表示所有标签个数
        self.n_char = 65535  # 所有字符的Unicode编码个数
        self.epsilon = 1e-100  # 无穷小量
        self.tag2idx = {'B-PER': 0,
                        'I-PER': 1,
                        'B-LOC': 2,
                        'I-LOC': 3,
                        'B-ORG': 4,
                        'I-ORG': 5,
                        'O': 6}
        self.idx2tag = dict(zip(self.tag2idx.values(), self.tag2idx.keys()))
        self.A = np.zeros((self.n_tag, self.n_tag))  # 转移概率矩阵,shape:7*7
        self.B = np.zeros((self.n_tag, self.n_char))  # 发射概率矩阵,shape:7*字的个数
        self.pi = np.zeros(self.n_tag)  # 初始隐状态概率,shape：4

    def train(self, corpus_path):
        """
        函数说明： 训练HMM模型, 得到模型参数pi,A,B

        Parameter：
        ----------
            corpus_path - 语料库的位置
        Return:
        -------
            None
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 13:42:50
        """
        with open(corpus_path, mode='r', encoding='utf-8') as fr:
            lines = fr.readlines()
        print('开始训练数据：')
        for i in tqdm(range(len(lines))):
            if len(lines[i]) == 1:
                continue
            else:
                cur_char, cur_tag = lines[i].split()
                self.B[self.tag2idx[cur_tag]][ord(cur_char)] += 1
                if len(lines[i - 1]) == 1:
                    self.pi[self.tag2idx[cur_tag]] += 1
                    continue
                pre_char, pre_tag = lines[i - 1].split()
                self.A[self.tag2idx[pre_tag]][self.tag2idx[cur_tag]] += 1
        self.pi[self.pi == 0] = self.epsilon  # 防止数据下溢,对数据进行对数归一化
        self.pi = np.log(self.pi) - np.log(np.sum(self.pi))
        self.A[self.A == 0] = self.epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))
        self.B[self.B == 0] = self.epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        np.savetxt('pi', self.pi)
        np.savetxt('A', self.A)
        np.savetxt('B', self.B)
        print('训练完毕！')

    def viterbi(self, Obs):
        """
        函数说明： 使用viterbi算法进行解码

        Parameter：
        ----------
            Obs - 要解码的文本string
        Return:
        -------
            path - 最可能的隐状态路径
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 16:52:42
        """
        T = len(Obs)
        delta = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        psi = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        delta[0] = self.pi[:] + self.B[:, ord(Obs[0])]  # 初始化
        for i in range(1, T):
            temp = delta[i - 1].reshape(self.n_tag, -1) + self.A  # 这里运用到了矩阵的广播算法
            delta[i] = np.max(temp, axis=0)
            delta[i] = delta[i, :] + self.B[:, ord(Obs[i])]
            psi[i] = np.argmax(temp, axis=0)
        path = np.zeros(T)
        path[T - 1] = np.argmax(delta[T - 1])
        for i in range(T - 2, -1, -1):  # 回溯
            path[i] = int(psi[i + 1][int(path[i + 1])])
        return path

    def predict(self, Obs):
        """
        函数说明： 将文本进行命名实体识别

        Parameter：
        ----------
            Obs - 要识别的文本
        Return:
        -------
            None
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 20:53:23
        """
        T = len(Obs)
        path = self.viterbi(Obs)
        for i in range(T):
            print(Obs[i]+self.idx2tag[path[i]]+'_|', end='')





In [3]:
model = HMM_model()
model.train('corpus/BIO_train.txt')


开始训练数据：


100%|██████████| 2220537/2220537 [00:04<00:00, 507210.97it/s]


训练完毕！


In [7]:
s = '林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？' \
    '谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分'
s1='到了中国哈尔滨'
model.predict(s1)

到O_|了O_|中B-LOC_|国I-LOC_|哈I-LOC_|尔I-LOC_|滨I-LOC_|

In [10]:
print(model.A)
print(model.B)


[[-5.51365589e+00 -5.61702715e-02 -2.40034845e+02 -2.40034845e+02
  -2.40034845e+02 -2.40034845e+02 -2.98399134e+00]
 [-5.43172711e+00 -6.67433010e-01 -9.33706112e+00 -2.40694183e+02
  -7.60246007e+00 -2.40694183e+02 -7.29783505e-01]
 [-2.40763988e+02 -2.40763988e+02 -3.13587769e+00 -1.79340929e-01
  -8.10758314e+00 -2.40763988e+02 -2.11680064e+00]
 [-6.64874164e+00 -2.41066134e+02 -3.47918738e+00 -9.61872226e-01
  -6.66449000e+00 -2.41066134e+02 -5.37135734e-01]
 [-2.40190050e+02 -2.40190050e+02 -2.40190050e+02 -2.40190050e+02
  -7.09832702e+00 -4.43394884e-03 -5.62747527e+00]
 [-6.65128472e+00 -2.41591925e+02 -5.42805410e+00 -2.41591925e+02
  -7.96612012e+00 -2.80876513e-01 -1.43193040e+00]
 [-4.87611332e+00 -2.44703965e+02 -4.13370718e+00 -2.44703965e+02
  -4.71038702e+00 -2.44703965e+02 -3.31962064e-02]]
[[-240.03501539 -240.03501539 -240.03501539 ... -240.03501539
  -240.03501539 -240.03501539]
 [-240.69623622 -240.69623622 -240.69623622 ... -240.69623622
  -240.69623622 -240.6962

In [11]:
print(model.pi)

[-2.85900807e+00 -9.73424016e+00 -2.84227556e+00 -2.41091362e+02
 -2.66266679e+00 -9.73424016e+00 -2.05179421e-01]
